In [4]:
X = [[6.8, 123], [5.9, 106], [7.2, 130]]
y = [1, 0, 1]

In [5]:
import random
from collections import Counter

# Gini impurity
def gini(y):
    counts = Counter(y)
    impurity = 1
    for c in counts.values():
        p = c / len(y)
        impurity -= p ** 2
    return impurity

In [6]:
def split_data(X, y, feature, value):
    left_X, left_y, right_X, right_y = [], [], [], []
    for i in range(len(X)):
        if X[i][feature] <= value:
            left_X.append(X[i])
            left_y.append(y[i])
        else:
            right_X.append(X[i])
            right_y.append(y[i])
    return left_X, left_y, right_X, right_y

In [7]:
def best_split(X, y, features):
    best_feature, best_value, best_gini = None, None, float("inf")
    for f in features:
        values = set(x[f] for x in X)
        for v in values:
            lX, ly, rX, ry = split_data(X, y, f, v)
            if len(ly) == 0 or len(ry) == 0:
                continue
            g = (len(ly)/len(y))*gini(ly) + (len(ry)/len(y))*gini(ry)
            if g < best_gini:
                best_gini = g
                best_feature, best_value = f, v
    return best_feature, best_value

In [8]:
def build_tree(X, y, depth=0, max_depth=3):
    if len(set(y)) == 1 or depth == max_depth:
        return Counter(y).most_common(1)[0][0]

    features = random.sample(range(len(X[0])), int(len(X[0]) ** 0.5))
    f, v = best_split(X, y, features)

    if f is None:
        return Counter(y).most_common(1)[0][0]

    lX, ly, rX, ry = split_data(X, y, f, v)

    return {
        "feature": f,
        "value": v,
        "left": build_tree(lX, ly, depth+1),
        "right": build_tree(rX, ry, depth+1)
    }

In [9]:
def predict_tree(tree, x):
    if not isinstance(tree, dict):
        return tree
    if x[tree["feature"]] <= tree["value"]:
        return predict_tree(tree["left"], x)
    else:
        return predict_tree(tree["right"], x)


In [10]:
def bootstrap(X, y):
    idxs = [random.randint(0, len(X)-1) for _ in range(len(X))]
    return [X[i] for i in idxs], [y[i] for i in idxs]

def random_forest_train(X, y, n_trees=10):
    forest = []
    for _ in range(n_trees):
        Xb, yb = bootstrap(X, y)
        tree = build_tree(Xb, yb)
        forest.append(tree)
    return forest

def random_forest_predict(forest, x):
    votes = [predict_tree(tree, x) for tree in forest]
    return Counter(votes).most_common(1)[0][0]
